<a href="https://colab.research.google.com/github/dernameistegal/airbnb_price/blob/main/data_utils/munich/translate_listings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Translation of airbnb dataset into english

In [1]:
from google.colab import drive
import pandas as pd
import torch
drive.mount('/content/drive')
!git clone https://github.com/dernameistegal/airbnb_price.git

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'airbnb_price' already exists and is not an empty directory.


In [2]:
import sys
sys.path.append("/content/airbnb_price/custom_functions")
import general_utils as ut
device = ut.get_device()

cuda available: False ; cudnn available: True ; num devices: 0
Using device cpu


## Preliminary works
- add comments_en column with values None
- add lang column with values None
- replace na values in comments colum with "no text"

In [6]:
%%capture
!wget http://data.insideairbnb.com/germany/bv/munich/2021-12-24/data/listings.csv.gz

In [ ]:
listings = pd.read_csv("listings.csv.gz")

In [4]:
path = "/content/drive/MyDrive/Colab/airbnb/munich/listings.pickle"
listings = pd.read_pickle(path)

In [3]:
%%capture
!pip install git+https://github.com/neuml/txtai#egg=txtai[pipeline]
# remember to restart runtime

In [8]:
# todo
listings.description
listings.host_about
listings.name

0                        Deluxw-Apartm. with roof terrace
1                      Apartment Munich/East with sundeck
2                       City apartment next to Pinakothek
3       Fancy, bright central roof top flat and homeof...
4                   Own floor & bath, parking & breakfast
                              ...                        
4990                         Tiny House  "Holzofen Lodge"
4991            Haus mit Garten im grünen Münchner Westen
4992        Dein neues Zuhause in München -schönes Zimmer
4993    Modern 125Sqm 2BR apartment, carpark, Olympia ...
4994                     Amazing Duplex flat with balcony
Name: name, Length: 4995, dtype: object

In [49]:
listings[["description_en", "host_about_en", "name_en"]] = listings[["description", "host_about", "name"]]
listings["lang"] = listings["description"]
listings[["description_en", "host_about_en", "name_en", "lang"]] = None

In [41]:
keep_ids, _, _ = ut.train_val_test_split(listings.index, train_size=0.20)  
listings = listings.loc[keep_ids]

In [50]:
listings[["description", "host_about", "name"]] = listings[["description", "host_about", "name"]].fillna("no text")

In [51]:
from torch.utils.data import DataLoader, Dataset

class Translatedata(Dataset):
    def __init__(self, data, column):
        super().__init__()
        self.data = data
        self.column = column

    def __getitem__(self, key):
        return self.data[self.column].iloc[key], self.data.index[key]

    def __len__(self):
        return len(self.data)

In [52]:
dataset = Translatedata(listings, "description")
dataloader = DataLoader(dataset, batch_size=200)

In [53]:
# remove text garbage	
listings["description"] = listings["description"].str.replace("<.*?>", "")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [54]:
listings["description"].iloc[0]

'Das ist eine 3 Raum Wohnung, Küche mit Waschmaschine, Kaffeemaschine, Wasserkocher, Toster und allem, was auf den Bildern zu sehen ist, WLAN und TV in jedem Zimmer. Bis zur U-Bahn Forstenrieder Allee sind 4 Min. zu Fuß, dann 10 Min. bis zum Marienplatz. Neben der U-Bahn Station befinden sich REWE, Bowling, Ca\xadfé, dm-drogerie markt.Die Lage ist absolut ruhig. Bettwäsche, TücherМы говорим по русски!'

In [55]:
translate(listings["description"].iloc[0])

'This is a 3 room apartment, kitchen with washing machine, coffee maker, kettle, toster and everything that can be seen in the pictures, Wi-Fi and TV in each room. Up to the underground Forstenrieder avenue are 4 min. walk, then 10 min. to Marienplatz. Next to the underground station are REWE, bowling, ca-fé, dm-drogerie market.The location is absolutely quiet. Bed linen, towels говорим по русски!'

# Detect language and sort by it

In [56]:
# detect language
from txtai.pipeline import Translation
translate = Translation(batch=200)
with torch.no_grad():
    for i, (text, id) in enumerate(dataloader):
        text = list(text)
        try: 
            if listings.loc[id, "lang"].isnull().values.any():
                language = translate.detect(list(text))
                listings.loc[id, "lang"] = language
        except:
            continue
        last_id = id[-1]
        print(i)

0
1
2
3
4


In [57]:
listings = listings.sort_values(by="lang")

In [58]:
path = "/content/drive/MyDrive/Colab/airbnb/munich/listings.pickle"
listings.to_pickle(path)

In [59]:
listings["lang"].value_counts()

en    511
de    478
es      2
ru      2
hu      1
it      1
ja      1
nl      1
tr      1
zh      1
Name: lang, dtype: int64

# Translate text

In [5]:
#@title this is script.py
from txtai.pipeline import Translation
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Create translation model
translate = Translation(batch=50)

path = "/content/drive/MyDrive/Colab/airbnb/munich/listings.pickle"
listings = pd.read_pickle(path)

class Translatedata(Dataset):
    def __init__(self, data, column):
        super().__init__()
        self.data = data
        self.column = column

    def __getitem__(self, key):
        return self.data[self.column].iloc[key], self.data.index[key]

    def __len__(self):
        return len(self.data)

dataset = Translatedata(listings, "description")
dataloader = DataLoader(dataset, batch_size=50)


with torch.no_grad():
    for i, (text, id) in enumerate(dataloader):
        text = list(text)
        if listings.loc[id, dataset.column +"_en"].isnull().values.any():

            sourcefinder = listings.loc[id, "lang"].unique()
            if len(sourcefinder) > 1:
                  print(sourcefinder)
                  sourcefinder = None
            else:
                print(i)
                sourcefinder = sourcefinder[0]

            translation = translate(list(text), "en", source = sourcefinder)
            listings.loc[id, dataset.column +"_en"] = translation
        else:
          print("jumped", i)
        
        listings.to_pickle(path)

jumped 0
1


Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/284M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/778k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/328M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/795k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/771k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.26M [00:00<?, ?B/s]

In [15]:
# description batch size 50
!python script.py

['de']
Downloading: 100% 1.11k/1.11k [00:00<00:00, 708kB/s]
Downloading: 100% 284M/284M [00:07<00:00, 39.9MB/s]
Downloading: 100% 42.0/42.0 [00:00<00:00, 25.8kB/s]
Downloading: 100% 778k/778k [00:00<00:00, 2.48MB/s]
Downloading: 100% 750k/750k [00:00<00:00, 1.98MB/s]
Downloading: 100% 1.21M/1.21M [00:00<00:00, 3.27MB/s]
Traceback (most recent call last):
  File "script.py", line 42, in <module>
    translation = translate(list(text), "en", source = sourcefinder)
  File "/usr/local/lib/python3.7/dist-packages/txtai/pipeline/text/translation.py", line 85, in __call__
    outputs.extend(self.translate(chunk, language, target))
  File "/usr/local/lib/python3.7/dist-packages/txtai/pipeline/text/translation.py", line 166, in translate
    translated = model.generate(**tokens)
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/grad_mode.py", line 28, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py"

In [6]:
listings = pd.read_pickle(path)

In [8]:
listings

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,description_en,host_about_en,name_en,lang
987,15075840,https://www.airbnb.com/rooms/15075840,20211224070709,2021-12-24,3 rooms near underground station in Munich,"Das ist eine 3 Raum Wohnung, Küche mit Waschma...",NaN,https://a0.muscache.com/pictures/f09e2733-28de...,23457435,https://www.airbnb.com/users/show/23457435,Liubov,2014-11-07,"Munich, Bavaria, Germany",Ljubov,within an hour,100%,80%,f,https://a0.muscache.com/im/pictures/user/66f90...,https://a0.muscache.com/im/pictures/user/66f90...,Thalkirchen - Obersendling - Forstenried - Für...,4.0,4.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,NaN,Thalkirchen-Obersendling-Forstenried-Fürstenri...,NaN,48.097390,11.496600,Entire rental unit,Entire home/apt,7,NaN,1 bath,3.0,4.0,"[""Children\u2019s books and toys for ages 0-2 ...",$400.00,2,1000,2.0,2.0,1000.0,1000.0,2.0,1000.0,NaN,t,30,39,39,40,2021-12-24,3,0,0,2016-09-18,2018-06-21,5.00,5.00,4.33,4.00,5.00,4.33,5.00,NaN,f,1,1,0,0,0.05,"This is a 3 room apartment, kitchen with washi...",None,None,de
4554,51664503,https://www.airbnb.com/rooms/51664503,20211224070709,2021-12-24,Kurztrip nach MUC in gemütlicher Atmosphäre?,"Liebe Airbnb-ler, die Wohnung ist nicht nur se...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,397324729,https://www.airbnb.com/users/show/397324729,Nadine,2021-04-17,"Munich, Bavaria, Germany",no text,within an hour,100%,75%,f,https://a0.muscache.com/im/pictures/user/30b7e...,https://a0.muscache.com/im/pictures/user/30b7e...,NaN,0.0,0.0,"['email', 'phone']",t,t,NaN,Hadern,NaN,48.105810,11.485340,Private room in rental unit,Private room,2,NaN,1 shared bath,1.0,1.0,"[""Ethernet connection"", ""Room-darkening shades...",$49.00,1,1125,1.0,1.0,1125.0,1125.0,1.0,1125.0,NaN,t,22,52,82,352,2021-12-24,3,3,0,2021-08-23,2021-09-04,5.00,5.00,5.00,5.00,5.00,5.00,5.00,NaN,f,1,0,1,0,0.73,"Dear Airbnb-ler, the apartment is not only ver...",None,None,de
2326,32522524,https://www.airbnb.com/rooms/32522524,20211224070709,2021-12-24,Ideal für WG oder Praktikanten von der BUPO,Die Küche des Apartments ist voll ausgestattet...,Inder Nachbarschaft befinden sich viele Restau...,https://a0.muscache.com/pictures/4e328a33-41be...,205832270,https://www.airbnb.com/users/show/205832270,K1-Mysecondhome De,2018-07-29,"Munich, Bavaria, Germany",no text,within an hour,100%,99%,f,https://a0.muscache.com/im/pictures/user/4df8f...,https://a0.muscache.com/im/pictures/user/4df8f...,Ramersdorf - Perlach,44.0,44.0,"['email', 'phone', 'jumio', 'offline_governmen...",t,t,"Munich, Bayern, Germany",Ramersdorf-Perlach,NaN,48.111755,11.617298,Entire rental unit,Entire home/apt,4,NaN,1 bath,2.0,2.0,"[""Room-darkening shades"", ""Long term stays a

In [9]:
path = "/content/drive/MyDrive/Colab/airbnb/munich/listings_secure_save.pickle"
listings.to_pickle(path)

In [ ]:
len(listings["host_id"].unique())

6338

In [ ]:
unique_host_about = listings["host_about"].unique()
unique_host_about = pd.DataFrame(unique_host_about, columns=["host_about"])
unique_host_about["host_about_en"] = unique_host_about["host_about"]
unique_host_about["host_about_en"] = None
unique_host_about.to_pickle("/content/drive/MyDrive/Colab/airbnb/data/translations/host_about")

In [ ]:
#@title new script
from txtai.pipeline import Translation
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Create translation model
translate = Translation(batch=64)

path = "/content/drive/MyDrive/Colab/airbnb/data/translations/host_about"
listings = pd.read_pickle(path)

class Translatedata(Dataset):
    def __init__(self, data, column):
        super().__init__()
        self.data = data
        self.column = column

    def __getitem__(self, key):
        return self.data[self.column].iloc[key], self.data.index[key]

    def __len__(self):
        return len(self.data)

dataset = Translatedata(listings, "host_about")
dataloader = DataLoader(dataset, batch_size=64)

jump_value=0
exit_value=1000

with torch.no_grad():
    for i, (text, id) in enumerate(dataloader):
        text = list(text)
        if i < jump_value:
            print("jumped", i)
            continue
        if i > exit_value:
            break
        if listings.loc[id, dataset.column +"_en"].isnull().values.any():
            print(i)
            translation = translate(list(text), "en")
            listings.loc[id, dataset.column +"_en"] = translation
        else:
            print("already done in ", i)

        listings.to_pickle(path)

In [ ]:
# host about 
!python script.py

already done in  0
already done in  1
already done in  2
already done in  3
already done in  4
already done in  5
already done in  6
already done in  7
already done in  8
already done in  9
already done in  10
already done in  11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
Downloading: 100% 1.11k/1.11k [00:00<00:00, 817kB/s]
Downloading: 100% 286M/286M [00:06<00:00, 44.9MB/s]
Downloading: 100% 800k/800k [00:00<00:00, 2.13MB/s]
Downloading: 100% 787k/787k [00:00<00:00, 2.48MB/s]
Downloading: 100% 1.29M/1.29M [00:00<00:00, 3.44MB/s]
Downloading: 100% 42.0/42.0 [00:00<00:00, 35.4kB/s]
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147

In [ ]:
translated_ha = pd.read_pickle("/content/drive/MyDrive/Colab/airbnb/data/translations/host_about")

In [ ]:
translated_ha.index = translated_ha.host_about

In [ ]:
ha_dict = translated_ha["host_about_en"].to_dict()

In [ ]:
listings_mapped_translations = listings["host_about"].map(ha_dict)

In [ ]:
listings["host_about_en"] = listings_mapped_translations

In [ ]:
path = "/content/drive/MyDrive/Colab/airbnb/data/translations/listings"
listings.to_pickle(path)

In [ ]:
listings["description_en"][500:550]

7197     The apartment is very close to two subway line...
9894     In the accommodation 1210 apartment is a compl...
5478     Nice flat for a wonderful stay in Vienna. Near...
8453     Perfect to explore Vienna. It is a private acc...
4548     This attic apartment offers a pleasant holiday...
8457     Newly furnished 2-room apartment with Viennese...
1849     Dear travellers! Welcome to a stylish home in ...
9889     In March 2020 this house was opened with units...
1852     Near the river! The Danube is across the road,...
8455     Our apartment is comfortably furnished and can...
8456     Vienna is a beautiful city that invites many v...
5472     Close to the Danube Canal, very bright, quiet ...
1856     Daughter 19 abroad - her room is vacated and w...
3754     Are you looking for a cozy and bright accommod...
9884     SCROLL DOWN FOR ENGLISH DESCRIPTION PLEASE! <b...
9883     This first-class 36 m2 design apartment is cen...
7203     Bright apartment in a prime location with holi.